## Convert xlsx data to csv format 

In [11]:
import xlrd
import csv

def csv_from_excel(path, sheet, csv_name):
    wb = xlrd.open_workbook(path)
    sh = wb.sheet_by_name(sheet)
    csv_file = open(csv_name, 'w')
    wr = csv.writer(csv_file, quoting=csv.QUOTE_ALL)

    for rownum in range(sh.nrows):
        if sh.row_values(rownum)[0] == '':
            return
        else:
            wr.writerow(sh.row_values(rownum))
    csv_file.close()

In [12]:
csv_from_excel("./mergeddata.xlsx", "Sheet1", "mergeddata.csv")

In [15]:
import pandas as pd 
import numpy as np 

data = pd.read_csv("mergeddata.csv")
data

,Price,Bedrooms,Bathrooms,Size,Crime,Transit,Shopping,Health,Entertainment,Community,Diversity,Schools,Employment
0,3075000.0,2.0,3.0,2249.0,6.3,90.1,90.1,45.8,41.5,53.5,69.0,83.1,93.0
1,2999990.0,3.0,3.0,2749.0,0.7,45.1,79.6,21.1,73.2,98.6,76.8,51.4,98.6
2,2999900.0,4.0,4.0,3249.0,4.2,96.5,97.2,14.1,79.6,58.5,31.7,78.9,84.5
3,2875000.0,2.0,3.0,2249.0,6.3,90.1,90.1,45.8,41.5,53.5,69.0,83.1,93.0
4,2595000.0,2.0,2.0,1999.0,6.3,90.1,90.1,45.8,41.5,53.5,69.0,83.1,93.0
5,2558000.0,3.0,2.0,2499.0,0.7,45.1,79.6,21.1,73.2,98.6,76.8,51.4,98.6
6,2469000.0,3.0,4.0,3249.0,0.7,45.1,79.6,21.1,73.2,98.6,76.8,51.4,98.6
7,2359990.0,3.0,3.0,1799.0,4.2,96.5,97.2,14.1,79.6,58.5,31.7,78.9,84.5
8,2350000.0,3.0,3.0,1799.0,4.2,96.5,97.2,14.1,79.6,58.5,31.7,78.9,84.5
9,2350000.0,3.0,3.0,1799.0,4.2,96.5,97.2,14.1,79.6,58.5,31.7,78.9,84.5


a full pariswise construction of the current data will have 3540C2 = 6264030 entries (too much!)

## Experiment with pairwise comparison data

In [16]:
exp_data = data[:10]
exp_data

,Price,Bedrooms,Bathrooms,Size,Crime,Transit,Shopping,Health,Entertainment,Community,Diversity,Schools,Employment
0,3075000.0,2.0,3.0,2249.0,6.3,90.1,90.1,45.8,41.5,53.5,69.0,83.1,93.0
1,2999990.0,3.0,3.0,2749.0,0.7,45.1,79.6,21.1,73.2,98.6,76.8,51.4,98.6
2,2999900.0,4.0,4.0,3249.0,4.2,96.5,97.2,14.1,79.6,58.5,31.7,78.9,84.5
3,2875000.0,2.0,3.0,2249.0,6.3,90.1,90.1,45.8,41.5,53.5,69.0,83.1,93.0
4,2595000.0,2.0,2.0,1999.0,6.3,90.1,90.1,45.8,41.5,53.5,69.0,83.1,93.0
5,2558000.0,3.0,2.0,2499.0,0.7,45.1,79.6,21.1,73.2,98.6,76.8,51.4,98.6
6,2469000.0,3.0,4.0,3249.0,0.7,45.1,79.6,21.1,73.2,98.6,76.8,51.4,98.6
7,2359990.0,3.0,3.0,1799.0,4.2,96.5,97.2,14.1,79.6,58.5,31.7,78.9,84.5
8,2350000.0,3.0,3.0,1799.0,4.2,96.5,97.2,14.1,79.6,58.5,31.7,78.9,84.5
9,2350000.0,3.0,3.0,1799.0,4.2,96.5,97.2,14.1,79.6,58.5,31.7,78.9,84.5


In [29]:
columns = ["Price","Bedrooms","Bathrooms","Size","Crime","Transit","Shopping","Health","Entertainment","Community","Diversity","Schools","Employment"]
columns1 = []
columns2 = []

for col in columns:
    columns1.append(col+'1')
    columns2.append(col+'2')
    
print(columns1)
print(columns2)

combined_columns = columns1[1:] + columns2[1:]
combined_columns.append("Rank")
print(combined_columns)
print(len(combined_columns))

['Price1', 'Bedrooms1', 'Bathrooms1', 'Size1', 'Crime1', 'Transit1', 'Shopping1', 'Health1', 'Entertainment1', 'Community1', 'Diversity1', 'Schools1', 'Employment1']
['Price2', 'Bedrooms2', 'Bathrooms2', 'Size2', 'Crime2', 'Transit2', 'Shopping2', 'Health2', 'Entertainment2', 'Community2', 'Diversity2', 'Schools2', 'Employment2']
['Bedrooms1', 'Bathrooms1', 'Size1', 'Crime1', 'Transit1', 'Shopping1', 'Health1', 'Entertainment1', 'Community1', 'Diversity1', 'Schools1', 'Employment1', 'Bedrooms2', 'Bathrooms2', 'Size2', 'Crime2', 'Transit2', 'Shopping2', 'Health2', 'Entertainment2', 'Community2', 'Diversity2', 'Schools2', 'Employment2', 'Rank']
25


In [27]:
mat = exp_data.as_matrix()
print(mat.shape)

(10, 13)


In [28]:
from scipy.special import comb
comb(10,2)

45.0

In [63]:
mat = np.around(mat, decimals=2)
print(mat[0])
print(mat[1])
print(mat[2])
comb_test = np.asarray([int(1 if mat[0,0] > mat[1,0] else 0)])
print(comb_test)
print(mat[0,1:].shape)
print(comb_test.shape)
con = np.concatenate((mat[0, 1:], mat[1, 1:], mat[2, 1:], comb_test))
print(con)

[  3.07500000e+06   2.00000000e+00   3.00000000e+00   2.24900000e+03
   6.30000000e+00   9.01000000e+01   9.01000000e+01   4.58000000e+01
   4.15000000e+01   5.35000000e+01   6.90000000e+01   8.31000000e+01
   9.30000000e+01]
[  2.99999000e+06   3.00000000e+00   3.00000000e+00   2.74900000e+03
   7.00000000e-01   4.51000000e+01   7.96000000e+01   2.11000000e+01
   7.32000000e+01   9.86000000e+01   7.68000000e+01   5.14000000e+01
   9.86000000e+01]
[  2.99990000e+06   4.00000000e+00   4.00000000e+00   3.24900000e+03
   4.20000000e+00   9.65000000e+01   9.72000000e+01   1.41000000e+01
   7.96000000e+01   5.85000000e+01   3.17000000e+01   7.89000000e+01
   8.45000000e+01]
[1]
(12,)
(1,)
[  2.00000000e+00   3.00000000e+00   2.24900000e+03   6.30000000e+00
   9.01000000e+01   9.01000000e+01   4.58000000e+01   4.15000000e+01
   5.35000000e+01   6.90000000e+01   8.31000000e+01   9.30000000e+01
   3.00000000e+00   3.00000000e+00   2.74900000e+03   7.00000000e-01
   4.51000000e+01   7.96000000e

In [26]:
count = 0
for i in range(10):
    for j in range(i):
        print(i,j)
        count += 1
print(count)

1 0
2 0
2 1
3 0
3 1
3 2
4 0
4 1
4 2
4 3
5 0
5 1
5 2
5 3
5 4
6 0
6 1
6 2
6 3
6 4
6 5
7 0
7 1
7 2
7 3
7 4
7 5
7 6
8 0
8 1
8 2
8 3
8 4
8 5
8 6
8 7
9 0
9 1
9 2
9 3
9 4
9 5
9 6
9 7
9 8
45


In [53]:
new_mat = np.zeros(shape=(int(comb(mat.shape[0], 2)), mat.shape[1]*2-1))
print(new_mat.shape)
print(mat.shape)

count = 0 
for i in range(exp_data.shape[0]):
    for j in range(i):
        # calculate rank as entry 1 price > entry 2 price ? 1: 0
        new_mat[count] = np.concatenate((mat[i,1:], mat[j,1:], np.asarray([1 if mat[i,0] > mat[j,0] else 0])))  
        count += 1
        
print(new_mat[:5])
print(new_mat[:,24])

(45, 25)
(10, 13)
[[  3.00000000e+00   3.00000000e+00   2.74900000e+03   7.00000000e-01
    4.51000000e+01   7.96000000e+01   2.11000000e+01   7.32000000e+01
    9.86000000e+01   7.68000000e+01   5.14000000e+01   9.86000000e+01
    2.00000000e+00   3.00000000e+00   2.24900000e+03   6.30000000e+00
    9.01000000e+01   9.01000000e+01   4.58000000e+01   4.15000000e+01
    5.35000000e+01   6.90000000e+01   8.31000000e+01   9.30000000e+01
    0.00000000e+00]
 [  4.00000000e+00   4.00000000e+00   3.24900000e+03   4.20000000e+00
    9.65000000e+01   9.72000000e+01   1.41000000e+01   7.96000000e+01
    5.85000000e+01   3.17000000e+01   7.89000000e+01   8.45000000e+01
    2.00000000e+00   3.00000000e+00   2.24900000e+03   6.30000000e+00
    9.01000000e+01   9.01000000e+01   4.58000000e+01   4.15000000e+01
    5.35000000e+01   6.90000000e+01   8.31000000e+01   9.30000000e+01
    0.00000000e+00]
 [  4.00000000e+00   4.00000000e+00   3.24900000e+03   4.20000000e+00
    9.65000000e+01   9.72000000e

In [61]:
def get_partial_data_as_matrix(data, entry):
    return data[:entry].as_matrix()

def generate_pairwise_data(mat):
    new_mat = np.zeros(shape=(int(comb(mat.shape[0], 2)), mat.shape[1]*2-1))
    print(new_mat.shape)

    count = 0 
    for i in range(mat.shape[0]):
        for j in range(i):
            # calculate rank as entry 1 price > entry 2 price ? 1: 0
            new_mat[count] = np.concatenate((mat[i,1:], mat[j,1:], np.asarray([1 if mat[i,0] > mat[j,0] else 0])))  
            count += 1
    return new_mat

def save_to_csv(path, mat):
    np.savetxt(path, mat, delimiter=",")


In [66]:
pairwise_data = get_partial_data_as_matrix(data, data.shape[0])

pairwise_mat = generate_pairwise_data(pairwise_data)

print(pairwise_mat)

(6267570, 25)
[[  3.00000000e+00   3.00000000e+00   2.74900000e+03 ...,   8.31000000e+01
    9.30000000e+01   0.00000000e+00]
 [  4.00000000e+00   4.00000000e+00   3.24900000e+03 ...,   8.31000000e+01
    9.30000000e+01   0.00000000e+00]
 [  4.00000000e+00   4.00000000e+00   3.24900000e+03 ...,   5.14000000e+01
    9.86000000e+01   0.00000000e+00]
 ..., 
 [  1.00000000e+00   1.00000000e+00   4.50000000e+02 ...,   7.04000000e+01
    2.68000000e+01   0.00000000e+00]
 [  1.00000000e+00   1.00000000e+00   4.50000000e+02 ...,   2.11000000e+01
    6.13000000e+01   0.00000000e+00]
 [  1.00000000e+00   1.00000000e+00   4.50000000e+02 ...,   6.48000000e+01
    7.32000000e+01   0.00000000e+00]]


all non-pricing features are included + a rank feature at the end 

In [67]:
save_to_csv("pairwise_data.csv", pairwise_mat)

In [68]:
pairwise_data_test = get_partial_data_as_matrix(data, 100)

pairwise_mat_test = generate_pairwise_data(pairwise_data_test)

print(pairwise_mat_test)

save_to_csv("pairwise_data_test.csv", pairwise_mat_test)

(4950, 25)
[[  3.00000000e+00   3.00000000e+00   2.74900000e+03 ...,   8.31000000e+01
    9.30000000e+01   0.00000000e+00]
 [  4.00000000e+00   4.00000000e+00   3.24900000e+03 ...,   8.31000000e+01
    9.30000000e+01   0.00000000e+00]
 [  4.00000000e+00   4.00000000e+00   3.24900000e+03 ...,   5.14000000e+01
    9.86000000e+01   0.00000000e+00]
 ..., 
 [  2.00000000e+00   2.00000000e+00   9.99000000e+02 ...,   8.31000000e+01
    9.30000000e+01   0.00000000e+00]
 [  2.00000000e+00   2.00000000e+00   9.99000000e+02 ...,   5.14000000e+01
    9.86000000e+01   0.00000000e+00]
 [  2.00000000e+00   2.00000000e+00   9.99000000e+02 ...,   8.31000000e+01
    9.30000000e+01   0.00000000e+00]]


## Experiment with list-wise data